# Stock Ranking Model: Growth Potential vs. Long Term Stability

In [1]:
# Import necessary libraries
import wrds
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import yfinance as yf
import plotly.express as px


### Step 1. Data Collection

In [2]:
conn = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
compustat_data = conn.raw_sql("""
    SELECT gvkey, tic, datadate, at, lt, ebit, ni, revt, mkvalt, che, dvt
    FROM comp.funda
    WHERE indfmt='INDL' AND datafmt='STD' AND popsrc='D' AND consol='C'
    AND datadate >= '2018-01-01'
""")

# Preprocess Tickers for Consistency
compustat_data['tic'] = compustat_data['tic'].str.upper().str.strip()

### Step 2: Calculate Missing Financial Metrics and Altman Z-Score

In [4]:
#Calculate ROA (Return on Assets)
compustat_data['roa'] = compustat_data['ni'] / compustat_data['at']

# Calculate ROE (Return on Equity)
compustat_data['roe'] = compustat_data['ni'] / (compustat_data['at'] - compustat_data['lt'])

# Calculate Dividend Payout Ratio
compustat_data['payout_ratio'] = compustat_data['dvt'] / compustat_data['ni']

# Altman Z-Score Formula
compustat_data['z_score'] = (
    1.2 * (compustat_data['che'] - compustat_data['lt']) / compustat_data['at'] +  # Working Capital / Total Assets
    1.4 * (compustat_data['ni'] / compustat_data['at']) +                         # Retained Earnings / Total Assets
    3.3 * (compustat_data['ebit'] / compustat_data['at']) +                      # EBIT / Total Assets
    0.6 * (compustat_data['mkvalt'] / compustat_data['lt']) +                    # Market Value of Equity / Total Liabilities
    1.0 * (compustat_data['revt'] / compustat_data['at'])                       # Sales / Total Assets
)

# Handle Missing Values and Infinite Calculations
compustat_data = compustat_data.replace([np.inf, -np.inf], np.nan).dropna()

### Step 3. Fuzzy Matching with Yahoo Finance Tickers

In [5]:
yf_tickers = ["AAPL", "MSFT", "TSLA", "AMZN", "GOOG", "NVDA", "META", "BRK-B", "JNJ", "PG"]
compustat_data['best_match'] = compustat_data['tic'].apply(
    lambda x: process.extractOne(x, yf_tickers)[0] if pd.notnull(x) else None
)


### Step 4. Fetch Historical Data

In [6]:
data = {}
for stock in yf_tickers:
    data[stock] = yf.download(stock, start="2018-01-01", end="2023-12-31")

# Combine closing prices into one DataFrame
close_prices = pd.DataFrame({stock: data[stock]["Close"] for stock in yf_tickers})
returns = close_prices.pct_change()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


### Step 5. Feature Engineering

In [7]:
# Add financial metrics from WRDS
features = pd.DataFrame(index=yf_tickers)
features['Volatility'] = returns.std() * np.sqrt(252)
# Calculate Momentum
features['Momentum'] = (
    close_prices.rolling(50).mean().iloc[-1] / close_prices.rolling(200).mean().iloc[-1] - 1
).clip(lower=-0.5, upper=0.5)  # Cap Momentum values between -0.5 and 0.5

# Map WRDS metrics to Yahoo tickers
for metric in ['roa', 'roe', 'payout_ratio', 'z_score']:
    features[metric] = features.index.map(
        lambda x: compustat_data[compustat_data['best_match'] == x][metric].values[0]
    )


### Step 6. Predictive Modeling and Ranking

In [8]:
# Normalize data using a scaler for direct use
scaler = StandardScaler()
scaled_features = pd.DataFrame(
    scaler.fit_transform(features), 
    index=features.index, 
    columns=features.columns
)

# Prepare scaled features for predictive modeling
X = scaled_features
y = features['Momentum']  # Replace with your target metric if needed

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Pipeline for predictive modeling
model_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Reapply scaler in pipeline for consistency
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train model
model_pipeline.fit(X_train, y_train)

# Ensure ranking uses scaled features for consistency
features['Predicted Return'] = model_pipeline.predict(scaled_features)

# Weighted Score calculation with scaled metrics
features['Weighted Score'] = (
    0.5 * features['Predicted Return'] +
    0.2 * features['roe'] +  # These should be appropriately scaled
    0.2 * features['z_score'] +
    0.1 * features['payout_ratio']
)

# Rank stocks based on Weighted Score
features['Rank'] = features['Weighted Score'].rank(ascending=False)

### Step 7. Visualizations

In [9]:
# Bar Chart for Rankings
fig = px.bar(
    features.sort_values(by='Rank'),  # Limit to top 10 for readability
    x=features.index,
    y='Weighted Score',
    title="Top 10 Stock Rankings by Weighted Score",
    labels={'index': 'Ticker', 'Weighted Score': 'Score'}
)
fig.show()

# Line Chart: Portfolio Backtest
top_stocks = features.sort_values(by='Rank').head(10).index
portfolio = close_prices[top_stocks].pct_change().mean(axis=1)
cumulative_return = (1 + portfolio).cumprod()

fig = px.line(
    cumulative_return,
    title="Cumulative Portfolio Return (Top Ranked Stocks)",
    labels={'value': 'Cumulative Return', 'index': 'Date'}
)
fig.show()

# Heatmap: Feature Correlations
fig = px.imshow(
    features.corr(),
    title="Feature Correlation Matrix",
    labels=dict(x="Metrics", y="Metrics", color="Correlation")
)
fig.show()

1. Volatility:
Likely negatively correlated with Stability Metrics (e.g., z_score and payout_ratio).
Indicates that more volatile stocks tend to be less stable.

2. Momentum:
Correlated with Predicted Return and Growth Score.
Highlights that recent price performance might be a strong driver of growth potential.

3.ROA and ROE:
Likely correlated with Weighted Score and Predicted Return.
Shows that profitability metrics are significant in ranking or performance predictions.

Z-Score:
Positively correlated with payout_ratio and Stability Score.
Suggests that financially healthy companies tend to pay consistent dividends.

Predicted Return and Rank:
Strong correlation between Predicted Return, Weighted Score, and Rank.
Confirms that the ranking logic heavily relies on the predictions from the model.

### Long Term Stability or Growth Potential?

In [16]:
# Growth Potential Score
features['Growth Score'] = (
    0.6 * features['Predicted Return'].clip(lower=0) +  # Avoid negative predicted returns
    0.3 * features['roe'].clip(lower=0) +              # Ensure ROE is non-negative
    0.2 * features['Momentum'].clip(lower=0)           # Avoid negative momentum
)

# Long-Term Stability Score
features['Stability Score'] = (
    0.4 * features['z_score'] +          # Financial health
    0.4 * features['payout_ratio'] +     # Dividends
    0.2 * (1 - features['Volatility'])   # Low volatility indicates stability
)

# Normalize Scores
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
features[['Growth Score', 'Stability Score']] = scaler.fit_transform(features[['Growth Score', 'Stability Score']])

# Fix size for Scatter Plot: Ensure size values are non-negative
features['Positive Weighted Score'] = features['Weighted Score'].clip(lower=0) + 0.01  # Add small constant

# Create Scatter Plot
fig = px.scatter(
    features,
    x='Growth Score',
    y='Stability Score',
    text=features.index,  # Annotate with stock tickers
    title="Growth Potential vs. Long-Term Stability",
    labels={'x': 'Growth Score', 'y': 'Stability Score'},
    size='Positive Weighted Score',  # Use non-negative values for size
    color='Weighted Score'           # Keep color as the original weighted score
)
fig.show()


In [17]:
# Categorize stocks
features['Category'] = features.apply(
    lambda row: 'Growth-Focused' if row['Growth Score'] > row['Stability Score'] + 0.05
    else 'Stability-Focused' if row['Stability Score'] > row['Growth Score'] + 0.05
    else 'Balanced',
    axis=1
)

# Bar Chart: Distribution of Categories
fig = px.bar(
    features['Category'].value_counts(),
    title="Distribution of Stocks by Category",
    labels={'index': 'Category', 'value': 'Number of Stocks'}
)
fig.show()

With the average stability score being greater than the average growth score, this portfolio is leaning towards being set up for long-term stability

In [18]:
# Save features to CSV with enhancements
def save_features_to_csv(features, filename="features.csv"):
    # Create a copy to avoid modifying the original DataFrame
    features_copy = features.copy()

    # Rename columns for better readability
    features_copy.rename(
        columns={
            "roa": "ROA (Return on Assets)",
            "roe": "ROE (Return on Equity)",
            "payout_ratio": "Payout Ratio",
            "z_score": "Altman Z-Score",
            "Positive Weighted Score": "Adjusted Weighted Score",
        },
        inplace=True
    )

    # Add a "Top Stock" column
    features_copy["Top Stock"] = features_copy["Rank"] <= 10

    # Ensure Ticker is included as the index
    features_copy.index.name = "Ticker"

    # Write metadata and save the file
    with open(filename, "w") as file:
        file.write("# Stock Ranking Data - Generated on 2023-12-05\n")  # Add metadata
        features_copy.to_csv(file, index=True, float_format="%.2f")  # Save data with consistent decimal precision

# Call the function to save the DataFrame
save_features_to_csv(features)


In [19]:
print(features.columns)  # Check all columns in the DataFrame
print(features['Positive Weighted Score'].head())  # Check if the column exists and has valid data


Index(['Volatility', 'Momentum', 'roa', 'roe', 'payout_ratio', 'z_score',
       'Predicted Return', 'Weighted Score', 'Rank', 'Growth Score',
       'Stability Score', 'Positive Weighted Score', 'Category'],
      dtype='object')
AAPL    0.596902
MSFT    0.023335
TSLA    1.552009
AMZN    1.053478
GOOG    0.255306
Name: Positive Weighted Score, dtype: float64


In [13]:
conn.close()